Este código foi baseado em https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi

#ORGANIZAÇÃO DO DATASET

Para treinar Ireneo, iniciamos clonando os modelos do Tensorflow no Github, uma vez que utilizaremos um modelo pré-treinado para a realização de Transfer Learning

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

Usamos o comando protoc para compilar arquivos .proto presentes no diretório de detecção de objetos do Tensorflow

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

Modificamos o arquivo setup.py para ser direcionado à uma versão adequada dos modelos TensorFlow

In [ ]:
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # reescrevemos a versão dos modelos
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

Instalações diversas necessárias para o treinamento

In [ ]:
!pip install pyyaml==5.3
!pip install /content/models/research/
!pip install tensorflow==2.8.0

Execução de um script de testes dos modelos instalados, para verificar se está tudo funcional

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py


Criamos um diretório para depois extrair do .zip e armazenar as imagens e anotações do dataset de treinamento. Nesse momento, deve existir um arquivo images.zip contendo todas as imagens e anotações necessárias (esse arquivo é gerado a partir do *Label Studio*).

In [ ]:
!mkdir content

In [ ]:
!unzip -q images.zip -d /content/images/

Baixamos scripts auxiliares e os executamos para fazer a conversão das anotações no formato VOC para CSV, separando também as imagens e anotações em conjuntos de treinamento e validação

In [ ]:
! wget https://raw.githubusercontent.com/svenancio/ireneo/refs/heads/main/src/treinamento/create_csv.py
! wget https://raw.githubusercontent.com/svenancio/ireneo/refs/heads/main/src/treinamento/create_tfrecord.py

In [ ]:
# Create CSV data files and TFRecord files
!python3 create_csv.py

Convertemos os dados anotados de imagens em arquivos TFRecord, um formato binário eficiente usado pelo TensorFlow para carregar grandes quantidades de dados de treinamento. Ele lê anotações das imagens (como coordenadas de *bounding boxes* e rótulos de objetos) e converte cada imagem e suas anotações em um exemplo TFRecord, usando uma estrutura de dados personalizada para organizar as informações. O script também gera um arquivo labelmap.pbtxt que associa classes de objetos a identificadores numéricos, necessário para treinar modelos de detecção de objetos com o TensorFlow.

In [ ]:
!python3 create_tfrecord.py --csv_input=images/train_labels.csv --labelmap=labelmap.txt --image_dir=images/train --output_path=train.tfrecord
!python3 create_tfrecord.py --csv_input=images/validation_labels.csv --labelmap=labelmap.txt --image_dir=images/validation --output_path=val.tfrecord

Criamos variáveis para armazenar os caminhos e referências dos

In [ ]:
train_record_fname = '/content/train.tfrecord'
val_record_fname = '/content/val.tfrecord'
label_map_pbtxt_fname = '/content/labelmap.pbtxt'

#TREINAMENTO DO MODELO

Temos agora à disposição diferentes modelos de detecção de objetos em imagens. Escolhemos um deles, mas deixamos os demais à disposição.

In [ ]:
MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'ssd-mobilenet-v2-fpnlite-320': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    }
}

# Se necessário, escolha outro modelo a partir da lista acima
chosen_model = 'ssd-mobilenet-v2-fpnlite-320'

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

Criamos um diretório "mymodel" para armazenar os modelos pré-treinados, então os baixamos.

In [ ]:
%mkdir /content/models/mymodel/
%cd /content/models/mymodel/

# Faz o download do modelo pré-treinado escolhido, em termo dos pesos de suas redes neurais
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# download do arquivo de configuração do modelo pré-treinado escolhido
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

Configuramos os parâmetros de treinamento, como número de passos (quantidade de vezes que o modelo ajusta seus pesos) e tamanho do "batch" (Cada batch é composto por várias imagens, incluindo suas respectivas anotações, como coordenadas de caixas delimitadoras e classes de objetos. Durante o treinamento, o modelo processa um batch por vez e ajusta seus pesos com base no erro médio calculado sobre todas as amostras no batch)

In [ ]:
num_steps = 40000

if chosen_model == 'efficientdet-d0':
  batch_size = 4
else:
  batch_size = 16

Configuramos mais alguns diretórios para receber arquivos de configuração e checkpoints de treinamento. Então extraímos o número de classes definidas no dataset de treinamento

In [ ]:
pipeline_fname = '/content/models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = '/content/models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

num_classes = get_num_classes(label_map_pbtxt_fname)
print('Total de classes:', num_classes)


Criamos um arquivo de configuração customizado para o treinamento de um modelo de detecção de objetos, ajustando parâmetros específicos no arquivo de pipeline base. Primeiro, carregamos o conteúdo do arquivo base de configuração (pipeline_fname) e, usando expressões regulares, substituimos valores como o caminho do checkpoint de pré-treinamento, os arquivos TFRecord de treino e teste, o caminho do arquivo de mapeamento de classes (label_map_path), o tamanho do batch (batch_size), o número de passos de treino (num_steps) e o número de classes (num_classes). Dependendo do modelo escolhido, também fazemos ajustes específicos. Ao final, escrevemos as alterações em um novo arquivo de configuração (pipeline_file.config).

In [ ]:
import re

%cd /content/models/mymodel
print('escrevendo arquivo de configuração customizado')

with open(pipeline_fname) as f:
    s = f.read()

with open('pipeline_file.config', 'w') as f:
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .08', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .026666', s)

    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)


Verificamos uma última vez o arquivo de configuração customizado para verificar se está tudo correto

In [ ]:
!cat /content/models/mymodel/pipeline_file.config

Configuramos mais algumas variáveis para armazenar caminhos e facilitar a referência

In [ ]:
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = '/content/models/mymodel/pipeline_file.config'
model_dir = '/content/training/'

Carregamos neste momento um Tensorboard, que nos ajuda a acompanhar o processo de treinamento

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

**EXECUÇÃO DO TREINAMENTO POR TRANSFERÊNCIA (TRANSFER LEARNING)!**

Hora de tomar um café ou fazer uns alongamentos

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1

Quando terminar o treinamento, armazenamos o modelo treinado em outro diretório

In [ ]:
!mkdir /content/custom_model_lite
output_directory = '/content/custom_model_lite'

last_model_path = '/content/training'

!python /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}


Conversão de um modelo TensorFlow salvo em um arquivo de modelo otimizado para dispositivos móveis e embarcados no formato TFLite (TensorFlow Lite).

In [ ]:
# Convert exported graph file into TFLite model file
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/content/custom_model_lite/saved_model')
tflite_model = converter.convert()

with open('/content/custom_model_lite/detect.tflite', 'wb') as f:
  f.write(tflite_model)

# TESTE E ANÁLISE DO MODELO

Definimos uma função para detectar objetos em fotos separadas para treinamento, e comparar os resultados com os rótulos e "bounding boxes" cadastrados

In [ ]:
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=10, savepath='/content/results', txt_only=False):

  # Pega os nomes de arquivos de todas as imagens na pasta de teste
  images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

  # Carrega o mapa de rótulos na memória
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Carrega o modelo TensorFlow Lite na memória
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Obtem detalhes do modelo
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  # Seleciona aleatoriamente as imagens de teste
  images_to_test = random.sample(images, num_test_images)

  # Faz a detecção para cada imagem
  for image_path in images_to_test:

      # Carrega a imagem e redimensiona para o formato esperado [1xHxWx3]
      image = cv2.imread(image_path)
      image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      imH, imW, _ = image.shape
      image_resized = cv2.resize(image_rgb, (width, height))
      input_data = np.expand_dims(image_resized, axis=0)

      # Normaliza os valores de pixel se estiver usando um modelo de ponto flutuante (não quantizado)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Realiza a detecção executando o modelo com a imagem como entrada
      interpreter.set_tensor(input_details[0]['index'],input_data)
      interpreter.invoke()

      # Recupera os resultados da detecção
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Coordenadas da Bounding box dos objetos detectados
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # índices das classes dos objetos detectados
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # nível de confiança de objetos detectados

      detections = []

      # Passa por todas as detecções e desenha a caixa de detecção se a confiança estiver acima do limite mínimo
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Obtém as coordenadas da caixa delimitadora e desenha a caixa
              # O "interpreter" pode retornar coordenadas fora das dimensões da imagem; é necessário ajustá-las usando max() e min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))

              cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

              # Desenha o rótulo
              object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

              detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])


      # Todos os resultados foram desenhados na imagem, agora exibe a imagem
      if txt_only == False: # "text_only" controla se queremos exibir os resultados da imagem ou apenas salvá-los em arquivos .txt
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12,16))
        plt.imshow(image)
        plt.show()

      # Salva os resultados da detecção em arquivos .txt (para calcular mAP)
      elif txt_only == True:

        # Obtém os nomes dos arquivos e caminhos
        image_fn = os.path.basename(image_path)
        base_fn, ext = os.path.splitext(image_fn)
        txt_result_fn = base_fn +'.txt'
        txt_savepath = os.path.join(savepath, txt_result_fn)

        # Escreve os resultados no arquivo de texto
        # (Usando o formato definido em https://github.com/Cartucho/mAP, que facilita o cálculo de mAP)
        with open(txt_savepath,'w') as f:
            for detection in detections:
                f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

  return

Executa a função de detectar objetos nas imagens de teste

In [ ]:
PATH_TO_IMAGES='/content/images/test'
PATH_TO_MODEL='/content/custom_model_lite/detect.tflite'
PATH_TO_LABELS='/content/labelmap.txt'
min_conf_threshold=0.5   # limiar de confiança (modifique para 0.01 se você não ver nenhum resultado de detecção)
images_to_test = 10   # quantidade de imagens a serem detectadas

# executa a detecção de objetos
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test)

Verificação do nível geral de precisão do modelo

Usamos bibliotecas externas para calcular o índice mAP (mean Average Precision)

In [ ]:
%%bash
git clone https://github.com/Cartucho/mAP /content/mAP
cd /content/mAP
rm input/detection-results/*
rm input/ground-truth/*
rm input/images-optional/*
wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py

In [ ]:
!cp /content/images/test/* /content/mAP/input/images-optional # Copy images and xml files
!mv /content/mAP/input/images-optional/*.xml /content/mAP/input/ground-truth/  # Move xml files to the appropriate folder

In [ ]:
!python /content/mAP/scripts/extra/convert_gt_xml.py

In [ ]:
# Configura novamente variáveis para rodar a inferência, desta vez para salvar os resultados da detecção como arquivos .txt
PATH_TO_IMAGES='/content/images/test'   # Caminho para a pasta de imagens de teste
PATH_TO_MODEL='/content/custom_model_lite/detect.tflite'   # Caminho para o arquivo de modelo .tflite
PATH_TO_LABELS='/content/labelmap.txt'   # Caminho para o arquivo labelmap.txt
PATH_TO_RESULTS='/content/mAP/input/detection-results' # Pasta para salvar os resultados da detecção
min_conf_threshold=0.1   # Limite mínimo de confiança

# Usa todas as imagens na pasta de teste
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = len(image_list)

# Diz à função para apenas salvar os resultados e não exibir as imagens
txt_only = True

# Executamos novamente
print('Iniciando inferência em %d imagens...' % images_to_test)
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Inferência finalizada!')

Executamos a análise do mAP sobre todos os rótulos do modelo

In [ ]:
%cd /content/mAP
!python calculate_map_cartucho.py --labels=/content/labelmap.txt

# COPIANDO O MODELO TREINADO

Reunimos na mesma pasta os arquivos labelmap.txt .pbtxt e a configuração junto ao modelo, e criamos um arquivo .zip

In [ ]:
# Move labelmap and pipeline config files into TFLite model folder and zip it up
!cp /content/labelmap.txt /content/custom_model_lite
!cp /content/labelmap.pbtxt /content/custom_model_lite
!cp /content/models/mymodel/pipeline_file.config /content/custom_model_lite

%cd /content
!zip -r custom_model_lite.zip custom_model_lite

Finalmente obtemos o modelo treinado, que será embarcado no dispositivo discriminativo Ireneo

In [ ]:
from google.colab import files

files.download('/content/custom_model_lite.zip')

# Quantização do modelo

In [ ]:
# Get list of all images in train directory
image_path = '/content/images/train'

jpg_file_list = glob.glob(image_path + '/*.jpg')
JPG_file_list = glob.glob(image_path + '/*.JPG')
png_file_list = glob.glob(image_path + '/*.png')
bmp_file_list = glob.glob(image_path + '/*.bmp')

quant_image_list = jpg_file_list + JPG_file_list + png_file_list + bmp_file_list

In [ ]:
# A generator that provides a representative dataset
# Code modified from https://colab.research.google.com/github/google-coral/tutorials/blob/master/retrain_classification_ptq_tf2.ipynb

# First, get input details for model so we know how to preprocess images
interpreter = Interpreter(model_path=PATH_TO_MODEL) # PATH_TO_MODEL is defined in Step 7 above
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

import random

def representative_data_gen():
  dataset_list = quant_image_list
  quant_num = 300
  for i in range(quant_num):
    pick_me = random.choice(dataset_list)
    image = tf.io.read_file(pick_me)

    if pick_me.endswith('.jpg') or pick_me.endswith('.JPG'):
      image = tf.io.decode_jpeg(image, channels=3)
    elif pick_me.endswith('.png'):
      image = tf.io.decode_png(image, channels=3)
    elif pick_me.endswith('.bmp'):
      image = tf.io.decode_bmp(image, channels=3)

    image = tf.image.resize(image, [width, height])  # TO DO: Replace 300s with an automatic way of reading network input size
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

In [ ]:
# Initialize converter module
converter = tf.lite.TFLiteConverter.from_saved_model('/content/custom_model_lite/saved_model')

# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input tensors to uint8 and output tensors to float32
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32
tflite_model = converter.convert()

with open('/content/custom_model_lite/detect_quant.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Set up parameters for inferencing function (using detect_quant.tflite instead of detect.tflite)
PATH_TO_IMAGES='/content/images/test'   #Path to test images folder
PATH_TO_MODEL='/content/custom_model_lite/detect_quant.tflite'   #Path to .tflite model file
PATH_TO_LABELS='/content/labelmap.txt'   #Path to labelmap.txt file
min_conf_threshold=0.5   #Confidence threshold (try changing this to 0.01 if you don't see any detection results)
images_to_test = 10   #Number of images to run detection on

# Run inferencing function!
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test)

In [ ]:
# Need to remove existing detection results first
!rm /content/mAP/input/detection-results/*

# Set up variables for running inference, this time to get detection results saved as .txt files
PATH_TO_IMAGES='/content/images/test'   # Path to test images folder
PATH_TO_MODEL='/content/custom_model_lite/detect_quant.tflite'   # Path to quantized .tflite model file
PATH_TO_LABELS='/content/labelmap.txt'   # Path to labelmap.txt file
PATH_TO_RESULTS='/content/mAP/input/detection-results' # Folder to save detection results in
min_conf_threshold=0.1   # Confidence threshold

# Use all the images in the test folder
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = min(500, len(image_list)) # If there are more than 500 images in the folder, just use 500

# Tell function to just save results and not display images
txt_only = True

# Run inferencing function!
print('Starting inference on %d images...' % images_to_test)
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Finished inferencing!')

Starting inference on 85 images...
Finished inferencing!


In [ ]:
!cd /content/mAP

In [ ]:
!cd /content

In [ ]:
!rm -rf /content/mAP/outputs/

In [ ]:
!python calculate_map_cartucho.py --labels=/content/labelmap.txt